## 0. Goal 
Get the max_dur information for each bird, used for applying trained models for new data

In [1]:
import os, sys, importlib, librosa, glob, h5py, tqdm
from scipy.io import wavfile
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from joblib import Parallel, delayed
import random
import umap, hdbscan
from collections import Counter
import seaborn as sns
from torch.utils.data import Dataset, DataLoader
import torch
from matplotlib.colors import ListedColormap

plt.rcParams['pdf.fonttype'] = 42 

In [2]:
# import my utility script
cluster_script_path = '/home/zz367/ProjectsU/EphysMONAO/Jupyter/MatlabCodes/ZZ_callClustering/'
sys.path.insert(1, cluster_script_path)
import vae_goffinet, hopkins
importlib.reload(vae_goffinet)

<module 'vae_goffinet' from '/home/zz367/ProjectsU/EphysMONAO/Jupyter/MatlabCodes/ZZ_callClustering/vae_goffinet.py'>

In [3]:
# create a custom colormap for spectrogram
jet = plt.get_cmap('jet', 255)
# Extract jet colors and prepend black at the beginning
jet_colors = jet(np.linspace(0, 1, 255))
custom_colors = np.vstack([[0, 0, 0, 1], jet_colors])  # Black for 0, then jet
custom_cmap = ListedColormap(custom_colors)

## 1. Inputs

In [4]:
fd_z4 = '/mnt/z4'
fd_data = os.path.join(fd_z4, 'zz367', 'EphysMONAO', 'Analyzed', 'vaeWav')
birdIDs = ['pair5RigCCU29', 'pair4RigACU68', 'pair4RigBCU53', 'pair2RigBCU25']
# color limits when calculating spectrograms, depending on the audio amplitude, may differ between birds
clims = [[1.5,7], [1.5,7], [1.5,8.5], [1.5,8]]
# what syllable to analyze
syl = ['v']

In [5]:
# define parameters for spectrograms
X_SHAPE = [128, 128]
p = {
    'get_spec': vae_goffinet.get_specZZ, # spectrogram maker
    'max_dur': 1e9, # maximum syllable duration
    'min_freq': 250, # minimum frequency
    'max_freq': 7500, # maximum frequency, default 7500
    'num_freq_bins': X_SHAPE[0], # hard-coded
    'num_time_bins': X_SHAPE[1], # hard-coded
    'nperseg': 256, # FFT
    'noverlap': 176, # FFT, determines window overlap when calculating spectrograms
    'spec_min_val': 1.5, # minimum log-spectrogram value
    'spec_max_val': 7, # maximum log-spectrogram value
    'fs': 20000, # audio samplerate
    'mel': False, # frequency spacing, mel or linear
    'time_stretch': False, # stretch short syllables?
    'within_syll_normalize': False, # normalize spectrogram values on a # spectrogram-by-spectrogram basis
    'max_num_syllables': None, # maximum number of syllables per directory
    'sylls_per_file': 20, # syllable per file, not used
    'real_preprocess_params': ('min_freq', 'max_freq', 'spec_min_val', 'spec_max_val'), # tunable parameters
    'int_preprocess_params': ('nperseg','noverlap'), # tunable parameters
    'binary_preprocess_params': ('time_stretch', 'mel', 'within_syll_normalize'), # tunable parameters
    'window_length': 0.032,  # for continuous sliding: size of the spectrogram window, unit is sec
    'hop_length': 0.004  # for continuous sliding: how much to hop for successive spectrogram windows, unit is sec
}

## 2. Loop through birds, calculate spectrograms

In [6]:
for bi in range(0,len(birdIDs)):
# for bi in range(1):
    bd = birdIDs[bi]
    print(f'Processing for {bd}...')
    # change the color limit
    p['spec_min_val'] = clims[bi][0]
    p['spec_max_val'] = clims[bi][1]
    
    win_ms = p['nperseg']/p['fs']*1000
    hop_ms = (p['nperseg']-p['noverlap'])/p['fs']*1000
    print(f'Spectrogram window size (ms): {win_ms:.3f}')
    print(f'Spectrogram window hop (ms): {hop_ms:.3f}')
    ## given the current run a name
    run_name = f'spec_goffinet_nn_256_176'
    print(run_name)
    # save to specific folder
    temp = ''.join(syl)
    fd_save = os.path.join(fd_z4, 'zz367', 'EphysMONAO', 'Analyzed', 'vaeWav', bd, 'Spectrogram2', temp)
    # if not os.path.exists(fd_save):
    #     os.makedirs(fd_save)
    # fn_h5 = os.path.join(fd_save, f'{bd}.{temp}.{run_name}.h5')
    # fn_info = os.path.join(fd_save, f'{bd}.{temp}.{run_name}.info.csv')
    # print(fn_h5)
    # print(fn_info)

    ## grab all wav files
    fns_wav = sorted(glob.glob(os.path.join(fd_data, bd, 'Audio', '*', '*.wav')))
    fns_label = sorted(glob.glob(os.path.join(fd_data, bd, 'Audio', '*', '*.label.txt')))
    len(fns_wav)

    ## loop through files determine the max syllable duration
    print('Calculating max duration...')
    with Parallel(n_jobs=48, verbose=5) as parallel:
        dur_all = parallel(delayed(vae_goffinet.ZZ_getDur_v1)(fn, fs=p['fs'], syl=syl) for fn in fns_label)

    dur_flat = [item for sublist in dur_all for item in sublist]

    q = 0.995
    max_dur = np.quantile(dur_flat, q)
    print(f'Total number of syllables: {len(dur_flat)}')
    print(f'{bd} {q} quantile: {max_dur:.4f}')
       

Processing for pair5RigCCU29...
Spectrogram window size (ms): 12.800
Spectrogram window hop (ms): 4.000
spec_goffinet_nn_256_176
Calculating max duration...


[Parallel(n_jobs=48)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done  66 tasks      | elapsed:    3.5s
[Parallel(n_jobs=48)]: Done 192 tasks      | elapsed:    3.8s
[Parallel(n_jobs=48)]: Done 354 tasks      | elapsed:    4.4s
[Parallel(n_jobs=48)]: Done 552 tasks      | elapsed:    5.0s
[Parallel(n_jobs=48)]: Done 786 tasks      | elapsed:    5.8s
[Parallel(n_jobs=48)]: Done 1056 tasks      | elapsed:    6.7s
[Parallel(n_jobs=48)]: Done 1362 tasks      | elapsed:    7.6s
[Parallel(n_jobs=48)]: Done 1704 tasks      | elapsed:    8.6s
[Parallel(n_jobs=48)]: Done 1921 out of 1921 | elapsed:    9.3s finished


Total number of syllables: 23240
pair5RigCCU29 0.995 quantile: 0.2800
Processing for pair4RigACU68...
Spectrogram window size (ms): 12.800
Spectrogram window hop (ms): 4.000
spec_goffinet_nn_256_176
Calculating max duration...


[Parallel(n_jobs=48)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done  66 tasks      | elapsed:    0.3s
[Parallel(n_jobs=48)]: Done 288 tasks      | elapsed:    1.0s
[Parallel(n_jobs=48)]: Done 612 tasks      | elapsed:    2.1s
[Parallel(n_jobs=48)]: Done 1008 tasks      | elapsed:    3.5s
[Parallel(n_jobs=48)]: Done 1476 tasks      | elapsed:    5.0s
[Parallel(n_jobs=48)]: Done 2016 tasks      | elapsed:    6.8s
[Parallel(n_jobs=48)]: Done 2628 tasks      | elapsed:    8.8s
[Parallel(n_jobs=48)]: Done 3312 tasks      | elapsed:   11.0s
[Parallel(n_jobs=48)]: Done 4005 out of 4100 | elapsed:   13.3s remaining:    0.3s
[Parallel(n_jobs=48)]: Done 4100 out of 4100 | elapsed:   13.5s finished


Total number of syllables: 36359
pair4RigACU68 0.995 quantile: 0.3250
Processing for pair4RigBCU53...
Spectrogram window size (ms): 12.800
Spectrogram window hop (ms): 4.000
spec_goffinet_nn_256_176
Calculating max duration...


[Parallel(n_jobs=48)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done  66 tasks      | elapsed:    0.3s
[Parallel(n_jobs=48)]: Done 288 tasks      | elapsed:    0.8s
[Parallel(n_jobs=48)]: Done 612 tasks      | elapsed:    2.0s
[Parallel(n_jobs=48)]: Done 1008 tasks      | elapsed:    3.2s
[Parallel(n_jobs=48)]: Done 1476 tasks      | elapsed:    4.8s
[Parallel(n_jobs=48)]: Done 2016 tasks      | elapsed:    6.6s
[Parallel(n_jobs=48)]: Done 2528 out of 2623 | elapsed:    8.2s remaining:    0.3s
[Parallel(n_jobs=48)]: Done 2623 out of 2623 | elapsed:    8.4s finished


Total number of syllables: 16194
pair4RigBCU53 0.995 quantile: 0.3230
Processing for pair2RigBCU25...
Spectrogram window size (ms): 12.800
Spectrogram window hop (ms): 4.000
spec_goffinet_nn_256_176
Calculating max duration...


[Parallel(n_jobs=48)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done  66 tasks      | elapsed:    0.2s
[Parallel(n_jobs=48)]: Done 288 tasks      | elapsed:    1.0s
[Parallel(n_jobs=48)]: Done 936 tasks      | elapsed:    3.1s
[Parallel(n_jobs=48)]: Done 1728 tasks      | elapsed:    5.6s


Total number of syllables: 16452
pair2RigBCU25 0.995 quantile: 0.3300


[Parallel(n_jobs=48)]: Done 2366 out of 2461 | elapsed:    7.4s remaining:    0.3s
[Parallel(n_jobs=48)]: Done 2461 out of 2461 | elapsed:    7.5s finished
